In [1]:
from datetime import datetime, timedelta
import time, requests, pandas, lxml
from lxml import html

In [2]:
def format_date(date_datetime):
    date_timetuple = date_datetime.timetuple()
    date_mktime = time.mktime(date_timetuple)
    date_int = int(date_mktime)
    date_str = str(date_int)
    return date_str

def subdomain(symbol, start, end):
    format_url = "{0}/history?period1={1}&period2={2}"
    tail_url =  "&interval=div%7Csplit&filter=div&frequency=1d"
    url = format_url.format(symbol, start, end) + tail_url
    return url

def header(subdomain):
    hdrs = {"authority": "finance.yahoo.com",
                "method": "GET",
                "path": subdomain,
                "scheme": "https",
                "accept": "text/html,application/xhtml+xml",
                "accept-encoding": "gzip, deflate, br",
                "accept-language": "en-US,en;q=0.9",
                "cache-control": "no-cache",
                "cookie": "cookies",
                "dnt": "1",
                "pragma": "no-cache",
                "sec-fetch-mode": "navigate",
                "sec-fetch-site": "same-origin",
                "sec-fetch-user": "?1",
                "upgrade-insecure-requests": "1",
                "user-agent": "Mozilla/5.0"}
    return hdrs

def scrape_page(url, header):
    page = requests.get(url, headers=header)
    element_html = html.fromstring(page.content)
    table = element_html.xpath('//table')
    table_tree = lxml.etree.tostring(table[0], method='xml')
    panda = pandas.read_html(table_tree)
    return panda
     
def clean_dividends(symbol, dividends):
    index = len(dividends)
    dividends = dividends.drop(index-1)
    dividends = dividends.set_index('Date')
    dividends = dividends['Dividends']
    dividends = dividends.str.replace(r'\Dividend', '')
    dividends = dividends.astype(float)
    dividends.name = symbol
    return dividends

def div_output(symbol):
    start = format_date(datetime.today() - timedelta(days=9125))
    end   = format_date(datetime.today())
    sub = subdomain(symbol, start, end)
    hdrs = header(sub)
    base_url = "https://finance.yahoo.com/quote/"
    url = base_url + sub
    dividends = scrape_page(url, hdrs)
    return dividends #clean_dividends(symbol, dividends[0])

In [3]:
aapl=div_output('aapl')


In [4]:
aapl

[                                                Date  \
 0  *Close price adjusted for splits.**Adjusted cl...   
 
                                            Dividends  
 0  *Close price adjusted for splits.**Adjusted cl...  ]

In [5]:
apple=div_output('aapl')

In [6]:
apple

[                                                Date  \
 0  *Close price adjusted for splits.**Adjusted cl...   
 
                                            Dividends  
 0  *Close price adjusted for splits.**Adjusted cl...  ]